# Face Recognition

In [3]:
import numpy as np
import os
import tensorflow as tf
import keras
import keras.layers as tfl
from keras.models import model_from_json
import matplotlib.pyplot as plt

In [4]:
json_file = open('/home/abdalla/Datasets/Face/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('/home/abdalla/Datasets/Face/model.h5')

In [6]:
print(model.inputs)
print(model.outputs)

[<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_1')>]
[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'Bottleneck_BatchNorm')>]


In [7]:
def img_to_encoding(image_path, model):
    img = keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
    img = np.around(np.array(img) / 255.0, decimals=12)
    x_train = np.expand_dims(img, axis=0)
    embedding = model.predict_on_batch(x_train)
    return embedding / np.linalg.norm(embedding, ord=2)

In [48]:
database = {}
path = '/home/abdalla/Datasets/Face/people/'

for i in os.listdir(path):
    database[i.split('.')[0]] =  img_to_encoding(path+i, model)


In [49]:
def who_is_it(image_path, database, model):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.
    
    Inputs:
        image_path -- path to an image
        database -- database containing image encodings along with the name of the person on the image
        model -- your Inception model instance in Keras
    
    Returns:
        min_dist -- the minimum distance between image_path encoding and the encodings from the database
        identity -- string, the name prediction for the person on image_path
    """
    
    # Compute the target "encoding" for the image.
    encoding =  img_to_encoding(image_path, model)
        
    # Initialize "min_dist" to a large value
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current db_enc from the database.
        dist = tf.linalg.norm( tf.subtract(encoding, db_enc) )

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name.
        if dist < min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [52]:
who_is_it(f'/home/abdalla/Datasets/Face/images/camera_{5}.jpg', database, model)

it's arnaud, the distance is tf.Tensor(0.37389022, shape=(), dtype=float32)


(<tf.Tensor: shape=(), dtype=float32, numpy=0.37389022>, 'arnaud')

# DONE :D